In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.ensemble import RandomForestClassifier
from sklearn import datasets
from sklearn import metrics
from sklearn.model_selection import train_test_split
from ipywidgets import IntProgress
from IPython.display import display

p = IntProgress()
display(p)

train = pd.read_csv(r"C:\Users\hg\Desktop\MLfinalproject\train.csv")
train_label = pd.read_csv(r"C:\Users\hg\Desktop\MLfinalproject\train_label.csv")
test = pd.read_csv(r"C:\Users\hg\Desktop\MLfinalproject\test.csv")

train_null = train.isnull()

for i in range(train.shape[0]):
    if train_null["company"][i] == False and train_null["agent"][i] == True:
        train["agent"][i] = train["company"][i]

train_filled_value = train.fillna({"children": 0, "country": "PRT","agent":9.0})

test_null = test.isnull()

for i in range(test.shape[0]):
    if test_null["company"][i] == False and test_null["agent"][i] == True:
        test["agent"][i] = test["company"][i]

test_filled_value = test.fillna({"country": "PRT","agent":9.0})

IntProgress(value=0)

<ipython-input-1-46e996aa58ac>:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train["agent"][i] = train["company"][i]
<ipython-input-1-46e996aa58ac>:30: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test["agent"][i] = test["company"][i]


In [2]:
train_filled_value = train_filled_value.drop(["ID","hotel","is_canceled","company","adr","reservation_status","reservation_status_date"], axis=1)
test_filled_value = test_filled_value.drop(["ID","hotel","company"], axis=1)

df_alldata = pd.concat([train_filled_value, test_filled_value], axis=0)
df_alldata.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 119390 entries, 0 to 27858
Data columns (total 26 columns):
 #   Column                          Non-Null Count   Dtype  
---  ------                          --------------   -----  
 0   lead_time                       119390 non-null  int64  
 1   arrival_date_year               119390 non-null  int64  
 2   arrival_date_month              119390 non-null  object 
 3   arrival_date_week_number        119390 non-null  int64  
 4   arrival_date_day_of_month       119390 non-null  int64  
 5   stays_in_weekend_nights         119390 non-null  int64  
 6   stays_in_week_nights            119390 non-null  int64  
 7   adults                          119390 non-null  int64  
 8   children                        119390 non-null  float64
 9   babies                          119390 non-null  int64  
 10  meal                            119390 non-null  object 
 11  country                         119390 non-null  object 
 12  market_segment   

In [3]:
category_column=["ID","hotel","is_canceled","arrival_date_year","arrival_date_month","meal","country","market_segment",
                 "distribution_channel","is_repeated_guest","reserved_room_type","assigned_room_type","deposit_type",
                "agent","customer_type","reservation_status","reservation_status_date"]
numerical_column=["lead_time","arrival_date_week_number","arrival_date_day_of_month","stays_in_weekend_nights",
                 "stays_in_week_nights","adults","children","babies","previous_cancellations","previous_bookings_not_canceled",
                 "booking_changes","days_in_waiting_list","required_car_parking_spaces","total_of_special_requests"]

category_column_new=[]
numerical_column_new=[]
for i in df_alldata:
    if i in category_column:
        category_column_new.append(i)
    if i in numerical_column:
        numerical_column_new.append(i)
for j in category_column_new:
    df_alldata[j] = df_alldata[j].astype('str')
df_numerical = df_alldata.loc[:,numerical_column_new]
df_category = df_alldata.loc[:,category_column_new]
df_category = pd.get_dummies(df_category)
df2 = pd.concat( [df_category, df_numerical], axis=1 )

df_train=df2[:91531]
df_test=df2[91531:]
#df_test.info()

df_train_numerical=df_numerical[:91531]
df_test_numerical=df_numerical[91531:]
df_train_category=df_category[:91531]
df_test_category=df_category[91531:]

In [4]:
#Logisticfold交叉驗證 + normalization
from sklearn import preprocessing, linear_model
from sklearn.model_selection import KFold
x = df_train
x = np.array(x)
y = train["is_canceled"]
kf = KFold(n_splits=5)
best_accuracy = 0
for train_index , test_index in kf.split(x):
    X_train, X_test, y_train, y_test = x[train_index], x[test_index], y[train_index], y[test_index]
    logistic_regr = linear_model.LogisticRegression()
    logistic_regr.fit(X_train,y_train)
    y_iscanceled_predict = logistic_regr.predict(X_train)
    accuracy = logistic_regr.score(X_test,y_test)
    print("Logistic Regression Accuracy = ",accuracy*100,"%")
    if accuracy > best_accuracy:
        best_accuracy = accuracy
        best_logistic_regr = logistic_regr
print("Best Accuracy = ",best_accuracy*100,"%")

c:\users\hg\appdata\local\programs\python\python39\lib\site-packages\sklearn\linear_model\_logistic.py:762: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


Logistic Regression Accuracy =  85.00027311957174 %


c:\users\hg\appdata\local\programs\python\python39\lib\site-packages\sklearn\linear_model\_logistic.py:762: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


Logistic Regression Accuracy =  83.65563203321315 %


c:\users\hg\appdata\local\programs\python\python39\lib\site-packages\sklearn\linear_model\_logistic.py:762: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


Logistic Regression Accuracy =  80.05025674642194 %


c:\users\hg\appdata\local\programs\python\python39\lib\site-packages\sklearn\linear_model\_logistic.py:762: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


Logistic Regression Accuracy =  79.50398776357478 %
Logistic Regression Accuracy =  77.73953894897848 %
Best Accuracy =  85.00027311957174 %


c:\users\hg\appdata\local\programs\python\python39\lib\site-packages\sklearn\linear_model\_logistic.py:762: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [5]:
#logistic final accuracy
x_train_new = x
y_train_new = train["is_canceled"]
y_train_new = y_train_new.reset_index(drop=True)

y_pred_is_canceled = best_logistic_regr.predict(x_train_new)
count = 0

for i in range(y_pred_is_canceled.shape[0]):
    if y_pred_is_canceled[i] != y_train_new[i]:
        count += 1
log_accuracy = 1-count/y_pred_is_canceled.shape[0]
print("Logistic Accuracy = ",log_accuracy*100,"%")

Logistic Accuracy =  82.18090045995345 %


In [8]:
#RandomForest adr
from sklearn.ensemble import RandomForestRegressor 
x = df_train
y = train["adr"]
y = np.array(y)
X_train,X_test,y_train,y_test=train_test_split(x,y,test_size=0.2,random_state=0)
rfe = RandomForestRegressor(n_estimators = 100, random_state = 0)
rfe.fit(X_train,y_train)
y_adr_predict_train = rfe.predict(X_train)
y_adr_predict_test = rfe.predict(X_test)
a=rfe.score(X_test,y_test)
mse_rfe = np.mean((y_adr_predict_train - y_train) ** 2)
print("MSE = ",mse_rfe)
print(y_adr_predict_train)
print(y_adr_predict_test)
y_adr_predict = np.concatenate((y_adr_predict_train,y_adr_predict_test),axis=0)
print(y_adr_predict)

MSE =  169.03101722173048
[171.88640336  58.15428785  61.60432304 ... 150.03083603  98.02645162
  33.76355984]
[ 79.47938752  55.72162062  28.53174157 ...  39.18507066 113.8825197
  62.81636739]
[171.88640336  58.15428785  61.60432304 ...  39.18507066 113.8825197
  62.81636739]


In [9]:
traincolumn = ["is_canceled","arrival_date_year","arrival_date_month","arrival_date_day_of_month","stays_in_weekend_nights","stays_in_week_nights","adr"]
df_train = train[traincolumn]
df_train["is_canceled_predict"] = y_pred_is_canceled
df_train["adr_predict"] = y_adr_predict

date_list = []
month_dict = {"January":"01","February":"02","March":"03","April":"04","May":"05",
             "June":"06","July":"07","August":"08","September":"09","October":"10",
             "November":"11","December":"12"}
for i in range(df_train.shape[0]):
    data = f"{df_train['arrival_date_year'][i]}/{month_dict[df_train['arrival_date_month'][i]]}/{df_train['arrival_date_day_of_month'][i]:02}"
    date_list.append(data)
df_train["arrival_date"] = np.array(date_list)
df_train = df_train.drop(["arrival_date_year","arrival_date_month","arrival_date_day_of_month"],axis=1)
df_train.head()

<ipython-input-9-53cc57a97f91>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_train["is_canceled_predict"] = y_pred_is_canceled
<ipython-input-9-53cc57a97f91>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_train["adr_predict"] = y_adr_predict
<ipython-input-9-53cc57a97f91>:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs

,is_canceled,stays_in_weekend_nights,stays_in_week_nights,adr,is_canceled_predict,adr_predict,arrival_date
0,0,0,0,-6.305161,1,171.886403,2015/07/01
1,0,0,2,75.052227,0,58.154288,2015/07/01
2,0,0,2,74.546401,1,61.604323,2015/07/01
3,0,0,2,76.376288,1,-4.897003,2015/07/01
4,0,0,2,49.411647,1,58.639773,2015/07/01


In [10]:
adr_list = []
is_canceled_list = []
for i in range(df_train.shape[0]):
    adr_list.append(abs(df_train["adr"][i] - df_train["adr_predict"][i]))
    is_canceled_list.append(abs(df_train["is_canceled"][i] - df_train["is_canceled_predict"][i]))
print("adr Ein = ",sum(adr_list)/df_train.shape[0])
print("is_canceled Ein = ",sum(is_canceled_list)/df_train.shape[0])

adr Ein =  51.53166347006269
is_canceled Ein =  0.17819099540046543


In [11]:
stay_list = []
for i in range(df_train.shape[0]):
    stay = df_train["stays_in_weekend_nights"][i] + df_train["stays_in_week_nights"][i]
    stay_list.append(stay)

df_train["stay"] = np.array(stay_list)
df_train = df_train.drop(["stays_in_weekend_nights","stays_in_week_nights"],axis=1)

df_train = df_train.drop(["adr","is_canceled"],axis=1)

In [12]:
condition = df_train["is_canceled_predict"] == 0
df_train = df_train[condition]
df_train = df_train.reset_index(drop=True)

In [13]:
daily_revenue_list = []
for i in range(df_train.shape[0]):
    daily_revenue = df_train["adr_predict"][i] * df_train["stay"][i]
    daily_revenue_list.append(daily_revenue) 

df_train["daily_revenue"] = np.array(daily_revenue_list)
df_train = df_train.drop(["adr_predict","stay","is_canceled_predict"],axis=1)

In [14]:
drop = []
head = 0
for i in range(1,df_train.shape[0]):
    if df_train["arrival_date"][i] == df_train["arrival_date"][head]: 
        df_train["daily_revenue"][head] = df_train["daily_revenue"][head] + df_train["daily_revenue"][i]
        drop.append(i)
    else:
        head = i
submit = df_train.drop(drop)
submit = submit.reset_index(drop=True)
submit

<ipython-input-14-8081819b65db>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_train["daily_revenue"][head] = df_train["daily_revenue"][head] + df_train["daily_revenue"][i]


,arrival_date,daily_revenue
0,2015/07/01,17513.633905
1,2015/07/02,17767.839436
2,2015/07/03,14006.099859
3,2015/07/04,13561.983338
4,2015/07/05,15712.841011
...,...,...
635,2017/03/27,31594.324322
636,2017/03/28,24813.225109
637,2017/03/29,34411.007198
638,2017/03/30,41985.240859


In [15]:
submit["label"] = train_label["label"]
predict_list = []
Ein = 0
for i in range(submit.shape[0]):
    predict = submit["daily_revenue"][i]//10000
    predict_list.append(predict)
    #if submit["label"][i] != submit["predict"][i]:
        #Ein+=1
submit["predict"] = np.array(predict_list)
submit

,arrival_date,daily_revenue,label,predict
0,2015/07/01,17513.633905,2.0,1.0
1,2015/07/02,17767.839436,1.0,1.0
2,2015/07/03,14006.099859,1.0,1.0
3,2015/07/04,13561.983338,1.0,1.0
4,2015/07/05,15712.841011,1.0,1.0
...,...,...,...,...
635,2017/03/27,31594.324322,2.0,3.0
636,2017/03/28,24813.225109,1.0,2.0
637,2017/03/29,34411.007198,2.0,3.0
638,2017/03/30,41985.240859,3.0,4.0


In [16]:
Ein = 0
for i in range(submit.shape[0]):
    if submit["label"][i] != submit["predict"][i]:
        Ein+=1
print("Ein = ",Ein/submit.shape[0])

Ein =  0.60625


In [17]:
y_test_iscanceled_predict = best_logistic_regr.predict(df_test)
y_test_adr_predict_nn = rfe.predict(df_test)
testcolumn = ["arrival_date_year","arrival_date_month","arrival_date_day_of_month","stays_in_weekend_nights","stays_in_week_nights"]
df_test = test[testcolumn]
df_test["is_canceled_predict"] = y_test_iscanceled_predict
df_test["adr_predict"] = y_test_adr_predict_nn
df_test.head()

<ipython-input-17-8cec490efba9>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_test["is_canceled_predict"] = y_test_iscanceled_predict
<ipython-input-17-8cec490efba9>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_test["adr_predict"] = y_test_adr_predict_nn


,arrival_date_year,arrival_date_month,arrival_date_day_of_month,stays_in_weekend_nights,stays_in_week_nights,is_canceled_predict,adr_predict
0,2017,April,1,2,5,0,94.717230
1,2017,April,1,4,10,1,56.984753
2,2017,April,1,2,5,0,56.335180
3,2017,April,1,2,5,1,127.953691
4,2017,April,1,2,4,1,196.586114


In [18]:
date_list = []
month_dict = {"January":"01","February":"02","March":"03","April":"04","May":"05",
             "June":"06","July":"07","August":"08","September":"09","October":"10",
             "November":"11","December":"12"}
for i in range(df_test.shape[0]):
    data = f"{df_test['arrival_date_year'][i]}-{month_dict[df_test['arrival_date_month'][i]]}-{df_test['arrival_date_day_of_month'][i]:02}"
    date_list.append(data)

df_test["arrival_date"] = np.array(date_list)
df_test = df_test.drop(["arrival_date_year","arrival_date_month","arrival_date_day_of_month"],axis=1)
df_test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 27859 entries, 0 to 27858
Data columns (total 5 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   stays_in_weekend_nights  27859 non-null  int64  
 1   stays_in_week_nights     27859 non-null  int64  
 2   is_canceled_predict      27859 non-null  int64  
 3   adr_predict              27859 non-null  float64
 4   arrival_date             27859 non-null  object 
dtypes: float64(1), int64(3), object(1)
memory usage: 1.1+ MB


<ipython-input-18-762c1d70825e>:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_test["arrival_date"] = np.array(date_list)


In [19]:
stay_list = []
for i in range(df_test.shape[0]):
    stay = df_test["stays_in_weekend_nights"][i] + df_test["stays_in_week_nights"][i]
    stay_list.append(stay)

df_test["stay"] = np.array(stay_list)
df_test = df_test.drop(["stays_in_weekend_nights","stays_in_week_nights"],axis=1)

In [20]:
condition = df_test["is_canceled_predict"] == 0
df_test = df_test[condition]
df_test = df_test.reset_index(drop=True)

In [21]:
daily_revenue_list = []
for i in range(df_test.shape[0]):
    daily_revenue = df_test["adr_predict"][i] * df_test["stay"][i]
    daily_revenue_list.append(daily_revenue)

In [22]:
df_test["daily_revenue"] = np.array(daily_revenue_list)
df_test = df_test.drop(["adr_predict","stay","is_canceled_predict"],axis=1)

In [23]:
drop = []
head = 0
for i in range(1,df_test.shape[0]):
    if df_test["arrival_date"][i] == df_test["arrival_date"][head]: 
        df_test["daily_revenue"][head] = df_test["daily_revenue"][head] + df_test["daily_revenue"][i]
        drop.append(i)
    else:
        head = i
submit_test = df_test.drop(drop)
submit_test = submit_test.reset_index(drop=True)
submit_test

<ipython-input-23-29dd57138732>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_test["daily_revenue"][head] = df_test["daily_revenue"][head] + df_test["daily_revenue"][i]


,arrival_date,daily_revenue
0,2017-04-01,24782.820121
1,2017-04-02,25547.794867
2,2017-04-03,40373.800421
3,2017-04-04,18811.057160
4,2017-04-05,32673.631468
...,...,...
148,2017-08-27,45904.180385
149,2017-08-28,74166.442135
150,2017-08-29,35958.531128
151,2017-08-30,26866.767559


In [24]:
predict_list = []
for i in range(submit_test.shape[0]):
    predict = submit_test["daily_revenue"][i]//10000
    predict_list.append(predict)
submit_test["predict"] = np.array(predict_list)

In [25]:
nolabel = pd.read_csv("C:\\Users\\hg\\Desktop\\MLfinalproject\\test_nolabel.csv")
nolabel

,arrival_date
0,4/1/2017
1,4/2/2017
2,4/3/2017
3,4/4/2017
4,4/5/2017
...,...
148,8/27/2017
149,8/28/2017
150,8/29/2017
151,8/30/2017


In [26]:
nolabel["label"] = np.array(predict_list)
nolabel

,arrival_date,label
0,4/1/2017,2.0
1,4/2/2017,2.0
2,4/3/2017,4.0
3,4/4/2017,1.0
4,4/5/2017,3.0
...,...,...
148,8/27/2017,4.0
149,8/28/2017,7.0
150,8/29/2017,3.0
151,8/30/2017,2.0


In [27]:
nolabel.to_csv("C:\\Users\\hg\\Desktop\\MLfinalproject\\test_nolabel.csv",index=False)
nolabel["label"].value_counts()

3.0    52
4.0    41
2.0    36
5.0    15
6.0     4
1.0     3
7.0     2
Name: label, dtype: int64